In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 3 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Formule de quadrature
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Formule de quadrature

On cherche à approcher l’intégrale :

$$
I^{[a,b]}(f) = \int_a^b f(x){\mathrm d}x.
$$

## Formule simple

A partir d’une formule de quadrature à $n+1$ points définie sur un intervalle de référence, par exemple $[−1, 1]$ :

$$
I_{n}^{[−1, 1]}(f) = \sum_{k=0}^{n} f(x_k) \omega_k
$$

on peut obtenir une formule de quadrature à $n+1$ points directement sur l'intervalle $[a,b]$ :

$$
I_{n}^{[a, b]}(f) = \sum_{k=0}^{n} f(\tilde{x}_k) \tilde{\omega}_k,  \quad \tilde{x}_k = a + \frac{b-a}{2} (x_k+1), \quad \tilde{x}_k =  \frac{b-a}{2} \omega_k
$$

où les $n$ abscisses de quadrature sont les $\tilde{x}_k$ et les poids associés sont les $\tilde{x}_k$.

In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import newton_cotes
from scipy import interpolate
import plotly.io as pio
pio.templates.default = "seaborn"

**Exemple pour $f(x) = \cos(\frac{\pi}{2}x)$ sur l'intervalle $[0,1]$ avec les formules de Newton-Cotes**

In [ ]:
def f(x):
    return np.cos((np.pi/2)*x)

xplot = np.linspace(0,1,100)

fig = go.Figure(layout_title='Formule simple')
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name='f(x)', line_width=3, line_color='rgb(76,114,176)'))

n = np.arange(1,20)

for ni in n:
    xk = np.linspace(0, 1, ni+1)
    p = interpolate.KroghInterpolator(xk, f(xk))
    fig.add_trace(go.Scatter(visible=False, x=xk, y=f(xk), mode='markers', line_color='rgb(221,132,82)', marker_size=8, showlegend=False))
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=p(xplot), fill='tozeroy', line_color='rgb(221,132,82)', name="Intégrale <br> estimée"))

# Make plot visible for s=2
fig.data[1].visible = True
fig.data[2].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "Nb de points : "}, steps=steps)]

fig.update_layout(sliders=sliders, height=500)
fig.show()

**Résultats pour $f(x) = \cos(\frac{\pi}{2}x)$ avec les formules de Newton-Cotes**

In [ ]:
def f(x):
    return np.cos((np.pi/2)*x)

res_exa = 2/np.pi

n = np.arange(1, 40, 1)

a = 0.
b = 1.

for ni in n:
    wk, _ = newton_cotes(ni, equal=1)
    xk = np.linspace(a, b, ni+1)
    dx = (b - a) / ni
    quad = dx * np.sum(wk * f(xk))
    err = np.abs(quad - res_exa)/res_exa
    print(f"Nb de points : {ni+1:2d} --> erreur relative  : {err:20.15e}")

**Résultats pour $g(x) = \sqrt(x) \log(x)$ avec les formules de Newton-Cotes**

In [ ]:
def g(x):
    return  np.sqrt(x)*np.log(x)

res_exa = -4./9.

n = np.arange(1, 40, 1)

a = 1.e-20
b = 1.

for ni in n:
    wk, _ = newton_cotes(ni, equal=1)
    xk = np.linspace(a, b, ni+1)
    dx = (b - a) / ni
    quad = dx * np.sum(wk * g(xk))
    err = np.abs(quad - res_exa)/np.abs(res_exa)
    print(f"Nb de points : {ni+1:2d} --> erreur relative  : {err:20.15e}")

## Formules composées

Pour la formule composée, on subdivise l'intervalle $[a,b]$ en $m$ sous-intervalles $[t_j,t_{j+1}]$, $j\in\{0, m-1\}$, $a=t_0 < t_j \ldots, <t_m=b$, sur lesquels on va utiliser une formule simple:

$$
I_{m,n}^{[a, b]}(f) = \sum_{j=0}^{m-1} I_{m,n}^{[t_j,t_{j+1}]}(f)
$$

**Exemple pour $f(x) = cos(\frac{\pi}{2}x)$ pour $m=2$ avec les formules de Newton-Cotes**

In [ ]:
def f(x):
    return np.cos((np.pi/2)*x)

xplot = np.linspace(0,1,100)

fig = go.Figure(layout_title='Formule composée avec Newton-Cotes')
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name='f(x)', line_width=3, line_color='rgb(76,114,176)'))

n = np.arange(1,20)

m = 2
x = np.linspace(0, 1, m+1)

for ni in n: 
    #print("nb pts : ", ni+1)
    for j in range(m):
        xk = np.linspace(x[j], x[j+1], ni+1)
        ####print(xk)
        p = interpolate.KroghInterpolator(xk, f(xk))
        xplot = np.linspace(x[j],x[j+1],100)
        fig.add_trace(go.Scatter(visible=False, x=xk, y=f(xk), mode='markers', line_color='rgb(76,114,176)', marker_size=8, showlegend=False))
        fig.add_trace(go.Scatter(visible=False, x=xplot, y=p(xplot), fill='tozeroy', name=f"j={j}"))

# Make plot visible for n=1
for j in range(m):
    for iplot in range(2):
        #print(iplot+j*2+1)
        fig.data[iplot+j*2+1].visible = True
    
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==2*m*i+1 or el==2*m*i+2 or el==2*m*i+3 or el==2*m*i+4 
                                   for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "Nb de points : "}, steps=steps)]

fig.update_layout(sliders=sliders, height=500)

fig.show()

**Résultats pour $f(x) = cos(\frac{\pi}{2}x)$ avec les formules de Newton-Cotes**

In [ ]:
res_exa = 2/np.pi

s = 3
w, _ = newton_cotes(s-1, equal=1)
w = w/(s-1)
r = np.linspace(0, 1, s)
print(f"Nb de points par intervalle : {s}\n")

a = 0
b = 1

n = np.array((1, 10, 100, 500, 1000, 10000, 100000))

for i, ni in enumerate(n):
    x = np.linspace(a, b, ni+1)
    res = 0.
    for j in range(ni):
        h = x[j+1]-x[j]
        res +=  h * np.sum(w * f(x[j]+h*r))
    err = np.abs(res - res_exa)/res_exa
    print(f"Nb d'intervalles : {ni:6d} --> erreur relative  : {err:20.15e}")

**Résultats pour $g(x) = \sqrt(x) \log(x)$ avec les formules de Newton-Cotes**

In [ ]:
res_exa = -4./9.

s = 7
w, _ = newton_cotes(s-1, equal=1)
w = w/(s-1)
r = np.linspace(0, 1, s)
print(f"Nb de points par intervalle : {s}\n")

a = 1e-20
b = 1

n = np.array((1, 10, 100, 500, 1000, 10000, 100000))

for i, ni in enumerate(n):
    x = np.linspace(a, b, ni+1)
    res = 0.
    for j in range(ni):
        h = x[j+1]-x[j]
        res +=  h * np.sum(w * g(x[j]+h*r))
    err = np.abs(res - res_exa)/res_exa
    print(f"Nb d'intervalles : {ni:6d} --> erreur relative  : {err:20.15e}")